## Explore

In [1]:
#filename='9x9_10games'
#filename='gogod_9x9_games'
filename='KGS2001'
#filename='KGS_10games'
#filename='KGS_100games'

### TRAIN AND TEST ATTRIBUTES AND SIZES

In [6]:
import gzip
import struct
    
        
total_images=0
total_bytes=0
for s in ['train','test']:   
    with gzip.open(filename+'-'+s+'-labels-idx1-ubyte.gz') as bytestream:
        magic_number=bytestream.read(4)
        data=bytestream.read()
        bytes_label=len(data)
    
    with gzip.open(filename+'-'+s+'-vectors-idx4-ubyte.gz') as bytestream:
        magic_number=bytestream.read(4)
        num_images=struct.unpack('>I',bytestream.read(4))[0]
        size_1=struct.unpack('>I',bytestream.read(4))[0]
        size_2=struct.unpack('>I',bytestream.read(4))[0]
        size_3=struct.unpack('>I',bytestream.read(4))[0]
        data=bytestream.read()
        bytes_vector=len(data)
    total_images+=num_images
    total_bytes+=bytes_label+bytes_vector+24
    print('{}-labels. bytes_label:{}'.format(s,bytes_label))
    print('{} BYTES. bytes_vector:{} bytes_label:{}'.format(s.upper(),bytes_vector,bytes_label))
    print(s,num_images,size_1,size_2,size_3,bytes_label+bytes_vector,float(bytes_vector)/(size_1*size_2*size_3))
    
    if num_images==bytes_vector/(size_1*size_2*size_3):
        print('{} CORRECT'.format(s))
    else:
        print('{} INCORRECT'.format(s))
print('total_images',total_images,'total_bytes',total_bytes)


with open(filename + '.bin','rb') as bytestream:
    data=bytestream.read()
    bin_total_bytes=len(data)
    bin_total_images=len(data)/(size_1*size_2*size_3+2)
    print('bin',size_1,size_2,size_3,bin_total_bytes,bin_total_images)

if total_images==bin_total_images:
    print('CORRECT')
else:
    print('INCORRECT')
    

train-labels. bytes_label:736738
TRAIN BYTES. bytes_vector:531921948 bytes_label:736738
('train', 368367, 4, 19, 19, 532658686, 368367.0)
train CORRECT
test-labels. bytes_label:130018
TEST BYTES. bytes_vector:93870108 bytes_label:130018
('test', 65007, 4, 19, 19, 94000126, 65007.0)
test CORRECT
('total_images', 433374, 'total_bytes', 626658860)
('bin', 4, 19, 19, 626658804, 433374)
CORRECT
